# **Advanced Machine Learning - Assignment 1**

Let's define the training dataset import function, fetching directly from the Github repo.

In [1]:
import pandas as pd

def import_training_dataset():
  urlFeatures = 'https://github.com/AlbezJelt/AML_Assignment1/raw/main/data/X_train.csv'
  urlLabels = 'https://github.com/AlbezJelt/AML_Assignment1/raw/main/data/y_train.csv'
  trainset = pd.read_csv(urlFeatures)
  labelset = pd.read_csv(urlLabels)
  return trainset, labelset

trainset, labelset = import_training_dataset()

Now we can explore the pandas datasets:

In [6]:
trainset.head()

,Unnamed: 0,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Private_room,Entire_home/apt
0,0,40.71239,-73.95271,4,2,0.19,1,0,1,0
1,1,40696.00000,-73.91303,4,17,0.66,2,307,1,0
2,2,40.62707,-74.02817,3,1,0.04,1,87,0,0
3,3,40.77910,-73.98565,1,4,0.08,1,0,0,1
4,4,40.75777,-73.93509,1,0,0.00,1,358,1,0


In [5]:
labelset.head()

,Unnamed: 0,price
0,0,65
1,1,57
2,2,225
3,3,175
4,4,125
